In [16]:
import re

# Part 1: parse query to list of dictionaries
def parseQuery(query):
    res = []
    
    # sample:
    #        query = "class//stundent/mark" 
    #        ['class', '//', 'stundent', '/', 'mark']   
    queryArray = re.split("(//)",query)
    temp = []
    for tmp in queryArray:
        if tmp !="//":
            temp.extend(re.split("(/)",tmp))
        else:
            temp.append(tmp)
    temp = [ tmp.strip() for tmp in temp]
    queryArray = [ tmp for tmp in temp if tmp]
    

#     whether first query is doc
    firstQuery = queryArray[0]
    if 'doc' in firstQuery:
        queryArray = queryArray[1:]
        end = firstQuery.index('.')
        collection = firstQuery[5 : end]
        res.append({'collection': collection})
    else:
        res.append({})

#     parse the rest of queries
    for i in range(len(queryArray)):
        query = queryArray[i]
        if query == "/" or  query == "//":
            i += 1
            continue
            
        n = len(query)
        name = ''
        axis = ''
        predicate = ''
        index_pred = n
#         parse predicate
        if query[-1] == ']':
            index_pred = query.index('[')
            predicate = query[index_pred + 1 : -1]
            query = query[ : index_pred]
#         parse name and axis
        if "::" in query:
            index_split = query.index('::')
            axis = query[: index_split]
            name = query[index_split + 2 : index_pred]
        else:
            if i>0 and queryArray[i-1] == "//":
                axis = 'descendant'
            else:
                axis = 'child'
            name = query[: index_pred]
        dict_query = {'name' : name, 'axis' : axis, 'predicate': predicate}
        res.append(dict_query)
        i += 1
    return res

In [17]:
query = "class//stundent/mark"
print(parseQuery(query))

[{}, {'name': 'class', 'axis': 'child', 'predicate': ''}, {'name': 'stundent', 'axis': 'descendant', 'predicate': ''}, {'name': 'mark', 'axis': 'child', 'predicate': ''}]


In [20]:
query1 = 'doc("library.xml")/child::library/child::album/child::songs/child::song/child::title'
query2 = "doc('library.xml')//album[child::year>=1990 and child::year<=2000]/child::artists/child::artist[child::country='Indonesia']/child::name"

In [21]:
print(parseQuery(query1))
print(parseQuery(query2))

[{'collection': 'library'}, {'name': 'library', 'axis': 'child', 'predicate': ''}, {'name': 'album', 'axis': 'child', 'predicate': ''}, {'name': 'songs', 'axis': 'child', 'predicate': ''}, {'name': 'song', 'axis': 'child', 'predicate': ''}, {'name': 'title', 'axis': 'child', 'predicate': ''}]
[{'collection': 'library'}, {'name': 'album', 'axis': 'descendant', 'predicate': 'child::year>=1990 and child::year<=2000'}, {'name': 'artists', 'axis': 'child', 'predicate': ''}, {'name': 'artist', 'axis': 'child', 'predicate': "child::country='Indonesia'"}, {'name': 'name', 'axis': 'child', 'predicate': ''}]


In [ ]:
# query is a dicitonary
# nodes is a list of dataTreeNode
def singleQuery(nodes,query):
    new_nodes = getNodesFromAxisAndName(nodes, query["axis"], query["name"])
    
    res = []
    
    if query["predicate"]:
        for node in new_nodes:
            if complexPredicate(node,query['predicate']):
                res.append(node)
    
    return res            

In [ ]:
# nodes is a list of dataTreeNode 
# queries is a list of dictionaries, which is parsed queries

def wholeQuery(nodes,queries):
    for query in queries[1:]:
        nodes = singleQuery(nodes,query)
    
    return nodes

In [ ]:
# query is a Xpath string

def getQuery(rootnode, query):
    
    queries = parseQuery(query)
    nodes = [rootnode]
    
    return wholeQuery(nodes, queries)

In [35]:
# recover the original dicionaries of one dataTreeNode

def recoverNode(node):
    
    node_name = node.getName()
    
    node_dict = dict()
    node_dict[node_name] = dict()

    if node.haveChild():

        child_nodes = node.getChild()
        
        # child_dict is like { name1:[<name1>,<name1>], name2:[<name2>]}
        child_dict = dict()
        for child_node in child_nodes:
            child_name = child_node.getName()
            if child_name in child_dict:
                child_dict[child_name].append(child_node)
            else:
                child_dict[child_name] = [child_node]

        for child_name in child_dict:
            
            # child_name_nodes is like  [<name1>,<name1>]
            child_name_nodes = child_dict[child_name]

            tmp = []
            for child_name_node in child_name_nodes:
                tmp.append(recoverNode(child_name_node)[child_name])
                
            if len(tmp) == 1:
                node_dict[node_name][child_name] = tmp[0]
            else:
                node_dict[node_name][child_name] = tmp

    else:
        node_dict[node_name] = node.getValue()
        
    
    # return a dictionary
    return node_dict


In [24]:
# The input of buildTree is a dictionary
def buildTree(data):
    # get the name of the root node from the key of dictionary
    root_name = list(data.keys())[0]
    
    # initialize the tree node without parent
    root_node = jsonTreeNode(root_name, None )
    
    # modify the child/value attribute of the root node
    modifyNode(root_node, data[root_name])
    
    return root_node
    
    
#  The node may contain:

# 1. a basic element without structure , like a string/integer/float
#    -> this node is a leaf node
#    -> we need to modify node.value

# 2. a dictionary 
#    -> this node has child nodes
#    -> we need to modify node.child
# (1) {  child_name: [...] }, a group of siblings with the same name, eg: album, artist, song, genre
# (2) { child_name1, child_name2, ...}, a group of siblings with different names
# (3) a mix of (1) and (2), { child_name1:[...], childname2, childname3, ...}

def modifyNode(node, data):
    if isinstance(data,dict):
        setChild(node, data)
    else:
        node.setValue(data)

# data is a dictionary
def setChild(node, data):
    child_list = []
    for key in data:
        if isinstance(data[key],list):
            #there are multiple child nodes with the same name
            child_list.extend(sameName(parent = node,name = key ,data = data[key]))
        else:
            new_node = jsonTreeNode(key,node)
            modifyNode(new_node,data[key])
            child_list.append(new_node)
    node.setChild(child_list)
    
    
# "data" is a list 
def sameName(parent,name,data):
    res_list = []
    for ele in data:
        new_node = jsonTreeNode(name,parent)
        modifyNode(new_node,ele)
        res_list.append(new_node)
    return res_list


class jsonTreeNode:
    
    def __init__(self, name, parent = None):
        self.name = name
        self.parent = parent
        self.child = []
        self.value = None
        
    def __repr__(self):
        return f"<{self.name}>"
        
    # Set
    def setValue(self, value):
        self.value = value
    
    def setChild(self, child):
        self.child = child
    
    # Get
    def getName(self):
        return self.name
    
    def getParent(self):
        return self.parent
    
    def getChild(self):
        return self.child
    
    def getValue(self):
        return self.value
    
    def getSibling(self):
        return [ sibling for sibling in self.parent.child if sibling != self ]
    
    
    # Judgement -> True/False
    def isRoot(self):
        if self.parent:
            return False
        else:
            return True
    

    def haveChild(self):
        if self.child:
            return True
        else:
            return False
    
    def haveValue(self):
        if self.value is None:
            return False
        else:
            return True
    
    def haveSibling(self):
        if len(self.getSibling()) < 1:
            return False
        else:
            return True
    


In [25]:
library = {'library':{ 
                        'album': 
                                [

                                    {
                                     'title': 'Bua Hati',
                                     'artists': {'artist': [{'name': 'Anang Ashanty', 'country': 'Indonesia'},
                                                            {'name': 'Kris Dayanti', 'country': 'Indonesia'}]},
                                     'songs': {'song': [ {'title': 'Timang-Timang', 'duration': '5:13'},
                                                         {'title': 'Miliki Diriku', 'duration': '5:35'},
                                                         {'title': 'Bua Hati', 'duration': '5:07'}]},
                                     'genres': {'genre': ['Pop', 
                                                          'World']},
                                     'year': 1998},

                                    {'title': 'Separuh Jiwaku Pergi',
                                     'artists': {'artist': {'name': 'Anang Ashanty', 'country': 'Indonesia'}},
                                     'songs': {'song': [ {'title': 'Separuh Jiwaku Pergi', 'duration': '5:00'},
                                                         {'title': 'Belajarlah Untuk Cinta', 'duration': '5:23'},
                                                         {'title': 'Hujanpun Menangis', 'duration': '4:17'}]},
                                     'genres': {'genre': ['Pop', 
                                                          'World']},
                                     'year': 1998}
                                ]
                        }
            }

In [26]:
root = buildTree(library)
print(root)

<library>


In [36]:
res = recoverNode(root)
print(res)

{'library': {'album': [{'title': 'Bua Hati', 'artists': {'artist': [{'name': 'Anang Ashanty', 'country': 'Indonesia'}, {'name': 'Kris Dayanti', 'country': 'Indonesia'}]}, 'songs': {'song': [{'title': 'Timang-Timang', 'duration': '5:13'}, {'title': 'Miliki Diriku', 'duration': '5:35'}, {'title': 'Bua Hati', 'duration': '5:07'}]}, 'genres': {'genre': ['Pop', 'World']}, 'year': 1998}, {'title': 'Separuh Jiwaku Pergi', 'artists': {'artist': {'name': 'Anang Ashanty', 'country': 'Indonesia'}}, 'songs': {'song': [{'title': 'Separuh Jiwaku Pergi', 'duration': '5:00'}, {'title': 'Belajarlah Untuk Cinta', 'duration': '5:23'}, {'title': 'Hujanpun Menangis', 'duration': '4:17'}]}, 'genres': {'genre': ['Pop', 'World']}, 'year': 1998}]}}


In [37]:
res == library

True